In [49]:
import csv
from random import randint

In [50]:
genderTable={
    "Male":0,
    "Female":1
}
schoolTable={
    "CCDS":1,
    "EEE":2,
    "CoB (NBS)":3,
    "SoH":4,
    "WKW SCI":5,
    "CoE":6,
    "MAE":7,
    "SPMS":8,
    "SBS":9,
    "SSS":10,
    "ASE":11,
    "NIE":12,
    "ADM":13,
    "MSE":14,
    "LKCMedicine":15,
    "CCEB":16,
    "CEE":17,
    "HASS":18
}

In [51]:
class Student:
    gender=""
    school=""
    CGPA=""
    tutorialGroup=""
    ID=""
    name=""
    
    team=""
    def __init__(self,gender,school,CGPA,tutorialGroup,ID,name):
        self.gender=gender
        self.school=school
        self.CGPA=CGPA
        self.tutorialGroup=tutorialGroup
        self.name=name
        self.ID=ID
    def isDigit(self,target):
        return ord(target) in range(ord('0'),ord('9')+1)
    def getCGPA(self):
        res=0
        if len(self.CGPA)==1:
            return 100*int(self.CGPA)
        else:
            for i in self.CGPA:
                if self.isDigit(i):
                    res=res*10+ord(i)-ord('0')
            return res
    def getGender(self):
        return genderTable[self.gender]
    def getSchool(self):
        return schoolTable[self.school]
    def getTutorialGroup(self):
        res=0
        for i in self.tutorialGroup:
            if self.isDigit(i):
                res=res*10+ord(i)-ord('0')
        return res
    def display(self):
        print(self.tutorialGroup,self.school,self.CGPA,self.gender,self.team)

In [52]:
def countAvg(group:list[Student]):
    num=0
    avgGender=0
    avgCGPA=0
    for student in group:
        avgGender=avgGender+student.getGender()
        avgCGPA=avgCGPA+student.getCGPA()
    return (avgGender/10.0),(avgCGPA/10.0)

In [53]:
#Get distance

# def distance(targetGender:float,targetCGPA:float,presentGender:float,presentCGPA:float,candidate:Student,step:int):
#     newGender=(presentGender*step+candidate.getGender())/(step+1)
#     newCGPA=(presentCGPA*step+candidate.getCGPA())/(step+1)
#     distGender=newGender/targetGender-1
#     distCGPA=newCGPA/targetCGPA-1
#     return (distGender**2+distCGPA**2)**0.5,newGender,newCGPA

def distance(targetGender:float,targetCGPA:float,presentList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    presentGender=0
    presentCGPA=0
    schoolRecord=[0 for i in range(0,19)]
    maxSame=0
    for student in presentList:
        schoolRecord[student.getSchool()]+=1
        presentCGPA+=student.getCGPA()
        presentGender+=student.getGender()
    for school in schoolRecord:
        maxSame=max(maxSame,school)
    distGender=(presentGender/5.0)/targetGender-1
    distCGPA=(presentCGPA/5.0)/targetCGPA-1
    distSchool=(1/40.0)*(maxSame-1)*2.71828**maxSame

    return coefficientGender*distGender**2+coefficientCGPA*distCGPA**2+coefficientSchool*distSchool**2

def distanceGroup(groupList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    targetGender=0
    targetCGPA=0
    for student in groupList:
        targetGender+=student.getGender()
        targetCGPA+=student.getCGPA()
    targetGender/=5.0
    targetCGPA/=5.0
    groupDistance=0
    for i in range(0,10):
        teamList:Student=[]
        for j in range(5*i,5*i+5):
            teamList.append(groupList[j])
        groupDistance+=distance(targetGender,targetCGPA,teamList,coefficientGender,coefficientCGPA,coefficientSchool)
    return groupDistance


In [54]:
#Read file
studentList:list[list[Student]]=[[] for i in range(0,121)]
with open("records.csv","r",encoding="utf-8") as records:
    reader=csv.reader(records)
    flag=True
    for row in reader:
        if flag:
            flag=False
            continue
        present=Student(row[4],row[2],row[5],row[0],row[3],row[1])
        # print(row)
        # print(present.getTutorialGroup())
        # present.display()
        studentList[present.getTutorialGroup()].append(present)
# for tutorialGroup in studentList:
#     for student in tutorialGroup:
#         student.display()

In [55]:
# i=0
# for tutorialGroup in studentList:
#     if i==0:
#         i+=1
#         continue
#     print(tutorialGroup[0].getTutorialGroup(),distanceGroup(tutorialGroup))
#     i+=1

In [56]:
teamID=1
flag=True
for tutorialGroup in studentList:
    if flag:
        flag=False
        continue
    #Part by Li Yitong
    T=400.0
    f=0.99
    while T>0.01:
        p1=0
        p2=0
        while p1%5==p2%5:
            p1=randint(0,49)
            p2=randint(0,49)
        pre=-distanceGroup(tutorialGroup,20,1,0.5)
        #swap
        tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]

        aft=-distanceGroup(tutorialGroup,20,1,0.5)
        # if pre>aft:
        #     print((aft-pre)/T)
        if pre>aft and randint(0,1)<1-(2.71828**((aft-pre)/T)):
            # swap(tutorialGroup[p1],tutorialGroup[p2])
            tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]
        T*=f
    for i in range(0,10):
        for j in range(5*i,5*i+5):
            tutorialGroup[j].team=teamID
        teamID+=1
i=0
for tutorialGroup in studentList:
    for student in tutorialGroup:
        student.display()
        i+=1
        if i%5==0:
            print()

with open("output.csv","w",encoding="utf-8") as output:
    writer=csv.writer(output)
    writer.writerow(["Tutorial Group","Student ID","School","Name","Gender","CGPA"])
    for tutorialGroup in studentList:
        for student in tutorialGroup:
            writer.writerow([student.tutorialGroup,student.ID,student.school,student.name,student.gender,student.CGPA])
# for tutorialGroup in studentList:
#     print(len(tutorialGroup))

G-1 EEE 4.03 Male 1
G-1 ASE 4.52 Female 1
G-1 SBS 3.95 Female 1
G-1 CCDS 4.08 Female 1
G-1 CoB (NBS) 3.95 Female 1

G-1 MAE 4.12 Male 2
G-1 SSS 4.07 Female 2
G-1 EEE 3.88 Male 2
G-1 CCEB 4.14 Male 2
G-1 CoB (NBS) 4.09 Male 2

G-1 CCDS 4.03 Female 3
G-1 CCDS 4.02 Male 3
G-1 EEE 4.2 Male 3
G-1 SoH 4.09 Female 3
G-1 WKW SCI 4.15 Male 3

G-1 CoB (NBS) 4.03 Female 4
G-1 NIE 3.85 Male 4
G-1 SoH 4.19 Female 4
G-1 SSS 4.17 Female 4
G-1 SoH 4 Male 4

G-1 EEE 4.05 Female 5
G-1 EEE 4.03 Female 5
G-1 SPMS 4.12 Female 5
G-1 CCDS 4.06 Male 5
G-1 EEE 4.06 Male 5

G-1 CoB (NBS) 4.01 Male 6
G-1 SSS 4.48 Female 6
G-1 CoE 4.22 Female 6
G-1 SoH 4.2 Male 6
G-1 WKW SCI 4.01 Female 6

G-1 SoH 4.09 Male 7
G-1 EEE 4.06 Male 7
G-1 MAE 4.04 Female 7
G-1 MAE 4.1 Female 7
G-1 SSS 4.2 Female 7

G-1 MSE 4.11 Female 8
G-1 EEE 4.18 Female 8
G-1 CoB (NBS) 4.18 Female 8
G-1 CoE 4.12 Male 8
G-1 EEE 4.02 Female 8

G-1 EEE 4.11 Male 9
G-1 CoB (NBS) 4.08 Male 9
G-1 SSS 3.88 Female 9
G-1 CoB (NBS) 4.19 Female 9
G-1 CCDS 4.11